In [1]:
import ROOT
ROOT.ROOT.EnableImplicitMT()
ROOT.gInterpreter.Declare('''
using Vec_t = const ROOT::RVec<float>&;
float ComputeInvariantMass(Vec_t pt, Vec_t eta, Vec_t phi, Vec_t mass) {
    const ROOT::Math::PtEtaPhiMVector p1(pt[0], eta[0], phi[0], mass[0]);
    const ROOT::Math::PtEtaPhiMVector p2(pt[1], eta[1], phi[1], mass[1]);
    return (p1 + p2).M();
}
float SumpT(Vec_t pt) {
    return (pt[0] + pt[1]);
}

float At(Vec_t pt,int n) {
    return (pt[n]);
}
''')

Welcome to JupyROOT 6.24/00


True

In [2]:
Process={"DY":{'Files':'/eos/cms/store/group/phys_egamma/akapoor/ChargeMisID/newsamples2/*.root',
               'Type':'Bkg','Tree':'Events',
               'Xsec':1},
        }


Definitions={
    #"Electron_genmotherpid":"GenPart_pdgId[GenPart_genPartIdxMother[Electron_genPartIdx]]",
    #"Electron_deepjetnearjet":"Jet_btagDeepFlavB[Electron_jetIdx]",
    "GoodElectrons1":"(Electron_pt > 10) && ((Electron_sieie<0.011 && abs(Electron_eta)<=1.479) || (Electron_sieie<0.030 && abs(Electron_eta)>1.479))",
    "GoodElectrons2":"(Electron_mvaTTH > 0.8) &&(abs(Electron_dxy)< 0.05) && (Electron_hoe<0.1)&&(Electron_eInvMinusPInv>-0.04) && (Electron_sip3d<8)",
    "GoodElectrons3":"(abs(Electron_eta)< 1.479) && (Electron_convVeto) && (Electron_mvaFall17noIso_WPL==1) && (Electron_pfRelIso03_all < (0.4*Electron_pt)) && (Electron_lostHits==0)",
    "GoodElectrons":"GoodElectrons1 && GoodElectrons2 && GoodElectrons3",
    "NGoodElectrons":"Sum(GoodElectrons)",
    "MyElectron_pt":"Take(Electron_pt[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_eta":"Take(Electron_eta[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_phi":"Take(Electron_phi[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_mass":"Take(Electron_mass[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_charge":"Take(Electron_charge[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_genPartIdx":"Take(Electron_genPartIdx[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_jetIdx":"Take(Electron_jetIdx[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
    "MyElectron_pt1":"MyElectron_pt[0]",
    "MyElectron_eta1":"MyElectron_eta[0]",
    "MyElectron_phi1":"MyElectron_phi[0]",
    "MyElectron_mass1":"MyElectron_mass[0]",
    "MyElectron_charge1":"MyElectron_charge[0]",
    "MyElectron_genPartIdx1":"MyElectron_genPartIdx[0]",
    "MyElectron_jetIdx1":"MyElectron_jetIdx[0]",
    "MyElectron_motherpid1":"GenPart_pdgId[GenPart_genPartIdxMother[MyElectron_genPartIdx1]]",
    "MyElectron_deepjet1":"Jet_btagDeepFlavB[MyElectron_jetIdx1]",
    "MyElectron_pt2":"MyElectron_pt[1]",
    "MyElectron_eta2":"MyElectron_eta[1]",
    "MyElectron_phi2":"MyElectron_phi[1]",
    "MyElectron_mass2":"MyElectron_mass[1]",
    "MyElectron_charge2":"MyElectron_charge[1]",
    "MyElectron_genPartIdx2":"MyElectron_genPartIdx[1]",
    "MyElectron_jetIdx2":"MyElectron_jetIdx[1]",
    "MyElectron_motherpid2":"GenPart_pdgId[GenPart_genPartIdxMother[MyElectron_genPartIdx2]]",
    "MyElectron_deepjet2":"Jet_btagDeepFlavB[MyElectron_jetIdx2]",
    "Dielectron_mass":"ComputeInvariantMass(MyElectron_pt, MyElectron_eta, MyElectron_phi, MyElectron_mass)",    
}

Aliases={
    "elpt":"MyElectron_pt",
    "eleta":"MyElectron_eta",
    "elphi":"MyElectron_phi",
    "elmass":"MyElectron_mass",
    "elq":"MyElectron_charge",
    "elpt1":"MyElectron_pt1",
    "eleta1":"MyElectron_eta1",
    "elphi1":"MyElectron_phi1",
    "elmass1":"MyElectron_mass1",
    "elq1":"MyElectron_charge1",
    "elpt2":"MyElectron_pt2",
    "eleta2":"MyElectron_eta2",
    "elphi2":"MyElectron_phi2",
    "elmass2":"MyElectron_mass2",
    "elq2":"MyElectron_charge2",
}

Selections={
    "Sum(GoodElectrons) == 2":"Two or more good electrons",
    "MyElectron_motherpid1==23 && MyElectron_motherpid2==23":"Come from Z",
    "(MyElectron_deepjet1<0.3033) && (MyElectron_deepjet2<0.3033)":"DeepJet cuts on ele",
    "(Dielectron_mass<121) && (Dielectron_mass>61)":"Z mass cut",
    "Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter":"Quality1",
    "Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter":"Quality2",
    "Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter":"Quality3",
}


In [3]:
#for define in Definitions.keys():
#    print(define+":"+Definitions[define])

In [4]:
ROOT.gStyle.SetOptStat(0)

def Clone(h):
    htemp=h[:]
    h=[]
    for hi in htemp:
        h.append(hi.Clone())
    return h

def prepareevents(Process,Definitions,Selection):
    import ROOT
    PDs=[]
    for cont in Process.keys():
        Files=Process[cont]['Files']
        Tree=Process[cont]['Tree']
        df = ROOT.RDataFrame(Tree, Files)
        for define in Definitions.keys():
            df = df.Define(define,Definitions[define])
        for alias in Aliases.keys():
            df = df.Alias(alias,Aliases[alias])
        for sel in Selections.keys():
            df = df.Filter(sel,Selections[sel])
        PDs.append({'cont':cont,'Type':Process[cont]['Type'],'RDF':df,'Xsec':Process[cont]['Xsec']})
    return PDs

def savehist(h=[],col=[],op="l",norm=False):
    h=Clone(h)
    c = ROOT.TCanvas("c","example",600,700);
    #pad1 = ROOT.TPad("pad1","pad1",0,0.3,1,1);
    #pad1.SetBottomMargin(0)
    #pad1.Draw()
    #pad1.cd()
    #ROOT.gPad.SetLogy()
    legend=ROOT.TLegend(0.5,0.8,0.9,0.9)
    
    if norm: h[0].Scale(1/h[0].Integral());
    #h[0].GetXaxis().SetTitleSize(0.04)
    #h[0].GetYaxis().SetTitleSize(0.02)
    
    h[0].SetLineColor(col[0])
    h[0].Draw("hist")
    legend.AddEntry(h[0].GetName(), h[0].GetName(), op)
    if len(h)>1:
        for i,hi in enumerate(h):
            if i!=0:
                if norm: h[i].Scale(1/h[i].Integral());
                h[i].SetLineColor(col[i])
                h[i].Draw("histsame")
                legend.AddEntry(h[i].GetName(), h[i].GetName(), op)
    legend.Draw()
    legend.SetTextSize(0.02)
    #h[0].SetAxisRange(h[0].GetMinimum(), 0.2*h[0].GetMaximum(), "Y");
    if norm: c.SaveAs((h[0].GetTitle())+"_norm.pdf")
    if not norm: c.SaveAs((h[0].GetTitle())+".pdf")
    
    
def savehistwithratio(h=[],col=[],op="l"):
    h=Clone(h)
    if len(h)!=2:
        print("Only works with 2 hists")
        return 0;
    h1=h[0].Clone()
    h2=h[1].Clone()
    c1 = ROOT.TCanvas("c1","example",600,700);
    pad1 = ROOT.TPad("pad1","pad1",0,0.3,1,1);
    pad1.SetBottomMargin(0);
    pad1.Draw();
    pad1.cd();
    legend=ROOT.TLegend(0.5,0.8,0.9,0.9)
    
    h1.DrawCopy();
    h1.SetLineColor(col[0])
    legend.AddEntry(h1.GetName(), h1.GetName(), op)
    h2.Draw("same");
    h2.SetLineColor(col[1])
    legend.AddEntry(h2.GetName(), h2.GetName(), op)
    legend.Draw()
    legend.SetTextSize(0.04)
    
    c1.cd();
    
    pad2 = ROOT.TPad("pad2","pad2",0,0,1,0.3);
    pad2.SetTopMargin(0);
    pad2.Draw();
    pad2.cd();
    h1.Sumw2();
    h1.SetStats(0);
    h1.Divide(h2);
    #h1.SetMarkerStyle(21);
    h1.SetMarkerSize(1)
    h1.Draw("text0 ep");
    c1.cd();
    
    c1.SaveAs((h[0].GetTitle())+".pdf")
    
def savehistonlyratio(h=[],col=[],op="l"):
    h=Clone(h)
    ROOT.gStyle.SetPaintTextFormat("4.4f");
    if len(h)!=2:
        print("Only works with 2 hists")
        return 0;
    h1=h[0].Clone()
    h2=h[1].Clone()
    c1 = ROOT.TCanvas("c1","example",600,700);
    legend=ROOT.TLegend(0.5,0.8,0.9,0.9)
    
    h1.Sumw2();
    h1.SetStats(0);
    h1.Divide(h2)
    #h1.SetMarkerStyle(21);
    h1.SetMarkerSize(1)
    h1.Draw("text0 ep");
    c1.SaveAs((h[0].GetTitle())+"_ratio.pdf")
    
def BookHisto(df,Args,Branch):
    return df.Histo1D(Args, Branch)

In [5]:
PDs=prepareevents(Process,Definitions,Selections)

Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
#for PD in PDs:
#    PD['RDF'].Display({"NGoodElectrons","elpt","MyElectron_pt"}).Print()

In [7]:
hMeeSS = PDs[0]['RDF'].Filter("(elq1+elq2)!=0", "Two SS").Histo1D(("Dielectron_mass_SS", "Dielectron_mass;m_{ee} (GeV);Events", 10, 71, 111), "Dielectron_mass")
hMee = PDs[0]['RDF'].Histo1D(("Dielectron_mass_all", "Dielectron_mass;m_{ee} (GeV);Events", 10, 71, 111), "Dielectron_mass")


hemotherpid1 = PDs[0]['RDF'].Histo1D(("MyElectron_motherpid1", "MyElectron_motherpid1;MyElectron_motherpid1;Events", 50, -25, 25), "MyElectron_motherpid1")

helpt1SS = PDs[0]['RDF'].Filter("(elq1+elq2)!=0", "Two SS").Histo1D(("elpt1_SS", "elpt1;elpt1 (GeV);Events", 5, 10, 100), "elpt1")
helpt1 = PDs[0]['RDF'].Histo1D(("elpt1_all", "elpt1;elpt1 (GeV);Events", 5, 10, 100), "elpt1")

In [8]:
savehist([hMeeSS.Clone(),hMee.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],norm=True)

savehist([hemotherpid1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

savehistonlyratio([hMeeSS.Clone(),hMee.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

Info in <TCanvas::Print>: pdf file Dielectron_mass_norm.pdf has been created
Info in <TCanvas::Print>: pdf file MyElectron_motherpid1.pdf has been created
Info in <TCanvas::Print>: pdf file Dielectron_mass_ratio.pdf has been created


In [9]:
savehist([helpt1SS.Clone(),helpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],norm=True)
savehist([helpt1SS.Clone(),helpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

savehistonlyratio([helpt1SS.Clone(),helpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen])

Info in <TCanvas::Print>: pdf file elpt1_norm.pdf has been created
Info in <TCanvas::Print>: pdf file elpt1.pdf has been created
Info in <TCanvas::Print>: pdf file elpt1_ratio.pdf has been created


In [10]:
print(PDs[0]['RDF'].GetColumnNames())

{ "GoodElectrons1", "GoodElectrons2", "GoodElectrons3", "GoodElectrons", "NGoodElectrons", "MyElectron_pt", "MyElectron_eta", "MyElectron_phi", "MyElectron_mass", "MyElectron_charge", "MyElectron_genPartIdx", "MyElectron_jetIdx", "MyElectron_pt1", "MyElectron_eta1", "MyElectron_phi1", "MyElectron_mass1", "MyElectron_charge1", "MyElectron_genPartIdx1", "MyElectron_jetIdx1", "MyElectron_motherpid1", "MyElectron_deepjet1", "MyElectron_pt2", "MyElectron_eta2", "MyElectron_phi2", "MyElectron_mass2", "MyElectron_charge2", "MyElectron_genPartIdx2", "MyElectron_jetIdx2", "MyElectron_motherpid2", "MyElectron_deepjet2", "Dielectron_mass", "elpt", "eleta", "elphi", "elmass", "elq", "elpt1", "eleta1", "elphi1", "elmass1", "elq1", "elpt2", "eleta2", "elphi2", "elmass2", "elq2", "run", "luminosityBlock", "event", "btagWeight_CSVV2", "btagWeight_DeepCSVB", "CaloMET_phi", "CaloMET_pt", "CaloMET_sumEt", "nElectron", "Electron_deltaEtaSC", "Electron_dr03EcalRecHitSumEt", "Electron_dr03HcalDepth1TowerSum

In [11]:
PDs[0]['RDF'].Report().Print()

Two or more good electrons: pass=206007     all=4621815    -- eff=4.46 % cumulative eff=4.46 %
Come from Z: pass=59439      all=206007     -- eff=28.85 % cumulative eff=1.29 %
DeepJet cuts on ele: pass=58892      all=59439      -- eff=99.08 % cumulative eff=1.27 %
Z mass cut: pass=58609      all=58892      -- eff=99.52 % cumulative eff=1.27 %
Quality1  : pass=58592      all=58609      -- eff=99.97 % cumulative eff=1.27 %
Quality3  : pass=58569      all=58592      -- eff=99.96 % cumulative eff=1.27 %


In [12]:
#PDs[0]['RDF'].Display({"MyElectron_pt","MyElectron_motherpid1"}).Print()